In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col logpop, set log() for population data

In [2]:
d[:logpop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6_1 = "
data{
    int total_tools[10];
    real logpop[10];
    int society[10];
}
parameters{
    real a;
    real bp;
    vector[10] a_society;
    real<lower=0> sigma_society;
}
model{
    vector[10] mu;
    sigma_society ~ cauchy( 0 , 1 );
    a_society ~ normal( 0 , sigma_society );
    bp ~ normal( 0 , 1 );
    a ~ normal( 0 , 10 );
    for ( i in 1:10 ) {
        mu[i] = a + a_society[society[i]] + bp * logpop[i];
        mu[i] = exp(mu[i]);
    }
    total_tools ~ poisson( mu );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6.1",  model=m12_6_1, output_format=:mcmcchain);

Input data for cmdstan

In [4]:
m12_6_1_data = Dict("total_tools" => d[:total_tools], "logpop" => d[:logpop], "society" => d[:society]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_1_data, ProjDir, diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.1.stan' at line 15)
Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.1.stan' at line 15)


If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.but if this warning occurs often then your model may be either severely ill-conditioned or

Describe the draws

In [6]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                   Mean           SD        Naive SE       MCSE         ESS    
         lp__  927.695347000  3.223277944 0.0509644992 0.1057341867  929.318654
accept_stat__    0.939637481  0.083346872 0.0013178298 0.0024215913 1000.000000
   stepsize__    0.048153875  0.008255322 0.0001305281 0.0013217442   39.009752
  treedepth__    5.912000000  0.525666321 0.0083115143 0.0345283236  231.776458
 n_leapfrog__   75.976000000 35.561827637 0.5622818655 2.1007313989  286.567604
  divergent__    0.000000000  0.000000000 0.0000000000 0.0000000000         NaN
     energy__ -921.098468500  4.118316136 0.0651162956 0.1257675946 1000.000000
            a    1.085613483  0.781000826 0.0123487073 0.0228695759 1000.000000
           bp    0.262664927  0.084975810 0.0013435855 0.0024119983 1000.000000
  a_society.1   -0.204369512  0.250383230 0.0039589065 0.0064867240 1000.000000
  a_

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*